# ANOVA
### 집단이 3개 이상일 때, 집단 간 평균 차이가 있는지 검정

- factor: 요인. 집단을 구별하는 변수. ex) 성별, 국가
- level: 요인의 수준(집단). 각 집단을 의미. ex) factor가 성별이었다면, level은 남, 여

# One-way ANOVA
### 요인이 1개면서 수준이 3개 이상인 분산분석. (집단을 구별하는 변수가 1개)
유의수준(알파)이 0.05일 때,

- 귀무가설: 모든 집단의 평균은 동일하다. 평균 차이가 없다.
    - if p-value > 0.05, 통계적으로 의미 없음


- 대립가설: 적어도 한 집단의 평균이 다른 집단들의 평균과 다르다.
    - if p-value < 0.05, 통계적으로 유의미함

## 가정
- 독립성: 자료의 추출은 독립적으로 이루어짐
- 정규성: 자료의 모집단의 분포는 정규분포
- 등분산성: 모든 모집단의 모분산은 동일

#### 데이터 준비

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('./onewayanova.txt', sep='\t')
df

,A,B,C,D
0,25,45,30,54
1,30,55,29,60
2,28,29,33,51
3,36,56,37,62
4,29,40,27,73


In [3]:
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D'])
df_melt.rename(columns={'variable':'treatments'}, inplace=True)
df_melt.drop(columns=['index'], inplace=True)
df_melt.head()

,treatments,value
0,A,25
1,A,30
2,A,28
3,A,36
4,A,29


### 집단간 샘플 개수가 같은지 확인

In [4]:
df_melt.groupby('treatments').agg(len)

,value
treatments,
A,5
B,5
C,5
D,5


### 1. 정규성 확인
- Sahpiro test: if p-value > alpha, 정규성 만족(정규분포 따름)
- Q-Q plot으로 확인

In [5]:
import scipy.stats

In [6]:
for _tr in df_melt.treatments.unique():
    # stat: 검정통계치
    stat, p = scipy.stats.shapiro(df_melt['value'][df_melt['treatments']==_tr])
    print("Normality of group {0} -> statistics: {1:.2f}, p-value: {2:.2f}".format(_tr, stat, p))

Normality of group A -> statistics: 0.93, p-value: 0.61
Normality of group B -> statistics: 0.93, p-value: 0.57
Normality of group C -> statistics: 0.95, p-value: 0.76
Normality of group D -> statistics: 0.94, p-value: 0.70


In [7]:
# ax = sns.boxplot(x='treatments', y='value', data=df_melt, color='green')
# ax = sns.swarmplot(x='treatments', y='value', data=df_melt, color='red')
# plt.show()

### 2. 독립성 확인
- random sampling인지 확인

### 3. 등분산성 확인
- Levene test: if p-value > 0.05, 집단의 분산이 모두 같다. 즉, 등분산성 만족
- box test

In [8]:
scipy.stats.levene(
    df_melt['value'][df_melt['treatments']=='A'],
    df_melt['value'][df_melt['treatments']=='B'],
    df_melt['value'][df_melt['treatments']=='C'],
    df_melt['value'][df_melt['treatments']=='D'] )

LeveneResult(statistic=1.9219593192195938, pvalue=0.16673281219949276)

In [9]:
scipy.stats.bartlett(
    df_melt['value'][df_melt['treatments']=='A'],
    df_melt['value'][df_melt['treatments']=='B'],
    df_melt['value'][df_melt['treatments']=='C'],
    df_melt['value'][df_melt['treatments']=='D'] )

BartlettResult(statistic=5.687843565012841, pvalue=0.1278253399753447)

## one-way anova 첫번째 방법

In [10]:
from scipy.stats import stats

In [11]:
fvalue, pvalue = stats.f_oneway(df['A'], df['B'], df['C'], df['D'])
print(fvalue, pvalue)

17.492810457516338 2.639241146210922e-05


## one-way anova 두번째 방법

In [20]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [21]:
model = ols('value ~ C(treatments)', df_melt).fit()

In [22]:
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(treatments),3.0,3010.95,1003.650,17.49281,0.000026
Residual,16.0,918.00,57.375,NaN,NaN


## 사후분석(post-hocs)
- 대립가설을 채택할 때(평균이 차이가 있는 집단이 적어도 한 개 존재), 구체적으로 어떤 수준(level)들이 차이가 있는지 확인
- 반드시 동질성이 확보되어야함(Levene 방법 사용)

## Family-wise Error Rate
- 여러 개의 가설 검정을 할 때 적어도 하나의 가설에서 1종 오류가 발생할 가능성
- 가설검정을 많이 할 수록 FWER은 증가
- 유의수준 5%에서 가설 검정을 1번 할 때, 1종 오류가 발생하지 않을 확률은 95%. FWER = 100% - 95% = 5%
- 가설검정을 2번했을 때, 2번 모두 1종 오류가 발생하지 않을 확률은 95% × 95% = 90.25%. FWER = 9.75%
- 가설검정을 3번했을 때, 3번 모두 1종 오류가 발생하지 않을 확률은 95% × 95% × 95% = 85.74%. FWER = 14.26%

### 방법

#### 1. 집단별 표본의 수와 분산이 동일한 경우
    - ** Tukey's HSD (Honestly Significant Difference) - 비교 대상 표본수가 동일하여야함. 표본수가 동일한 경우 가장 많이 사용되는 사후검정 기법
    
    
#### 2. 집단별 표본의 수는 다르지만 분산의 동일한 경우
    - Scheffe - 지나치게 보수적으로 엄격하게 사후검정 수행. 통계적으로 유의미한 차이를 도출하기 쉽지 않으므로 잘 사용하지 않으나 2번에서는 추천
    - Fisher's LSD (Least Significant Difference) - 오차비율을 통제하지 않아 상대적으로 엄격하지 않은 기준으로 사후 검정(1종 오류 발생확률을 통제하지 않음)
    
    
#### 3. 집단별 표본의 수도 다르고 분산의 동질성도 확보되지 않은 경우
    - Games-Howell


---

** Bonferroni's correction
- 모수, 비모수에 모두 적용 가능
- 엄격함의 정보로 봤을 때, Tukey < Bonferroni < Scheffe
- 그러나 비교대상이 많을수록 검정력이 약해짐


### Bonferroni correction

In [15]:
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

In [16]:
comp = MultiComparison(df_melt.value, df_melt.treatments)

- if p-value < 0.05, 두 수준(집단)의 평균 차이만 유의함
- reject column으로 확인

In [17]:
result = comp.allpairtest(scipy.stats.ttest_ind, method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
A,B,-2.8918,0.0201,0.1209,False
A,C,-0.6375,0.5416,1.0,False
A,D,-7.2136,0.0001,0.0005,True
B,C,2.6015,0.0315,0.1893,False
B,D,-2.3837,0.0443,0.2658,False
C,D,-6.8767,0.0001,0.0008,True


### 집단별 표본의 수와 분산이 동일한 경우 - Tukey's HSD

In [18]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [19]:
hsd = pairwise_tukeyhsd(df_melt['value'], df_melt['treatments'], alpha=0.05)
hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
A,B,15.4,0.0251,1.6929,29.1071,True
A,C,1.6,0.9,-12.1071,15.3071,False
A,D,30.4,0.001,16.6929,44.1071,True
B,C,-13.8,0.0482,-27.5071,-0.0929,True
B,D,15.0,0.0296,1.2929,28.7071,True
C,D,28.8,0.001,15.0929,42.5071,True


### Tukey's HSD 두번째 방법

In [20]:
sp.posthoc_tukey_hsd(df_melt.value, df_melt.treatments)

NameError: name 'sp' is not defined

### 집단별 표본의 수는 다르지만 분산의 동일한 경우 - Scheffe

In [ ]:
import scikit_posthocs as sp

In [ ]:
# 각각의 값이 집단간 p-value
# p-value < 0.05인 집단은 평균의 차이가 있음이 통계적으로 유의함
sp.posthoc_scheffe(df_melt, val_col='value', group_col='treatments')